### Tutorial 4 Pivot tables and plots

#### Creating yearly data by aggregating monthly data

In [3]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
df_sales_volume_data = pd.read_csv('complete_dataset.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'complete_dataset.csv'

In [ ]:
df_sales_volume_data

#### First, we create a pivot table, where we aggregate the sum of the number of items sold per product and year. When there are no iems sold, we replace with zero instead of missing value.

#### Then we set margins to true to create totals for rows and columns

In [ ]:
pivot = pd.pivot_table(df_sales_volume_data,index=["Year"],values=["Number_of_items"],
    columns=["Prodno"],aggfunc='sum',fill_value=0, margins=True)

In [ ]:
pivot

#### We convert the pivot to dataframe so it is easier to work with

In [ ]:
df_pivot=pd.DataFrame(pivot)

In [ ]:
df_pivot

#### We drop the column 'All' that sums up each row and keep the row 'All' since we need it to sort top sold products

In [ ]:
df_pivot.columns=df_pivot.columns.droplevel() 
df_pivot.drop('All', axis=1,inplace=True) 
df_pivot

#### 1) We select the top 10 products and sort them. 
#### 2) We create a list with the top 10 sold products
#### 3) We drop 'All' row since we dont need it anymore to make the selection
#### 4) We slice the pivot_df and capture only the 10 top sold products

In [ ]:
prod_list=pd.DataFrame(df_pivot.loc['All'].sort_values(ascending=False).head(10))
prod_list=prod_list.index.tolist()
df_pivot.drop('All',inplace=True)
df_top_prod=df_pivot[prod_list] 

In [ ]:
prod_list

#### Finally, we plot the yearly demand for the top 10 products

In [ ]:
(df_top_prod).plot(figsize=(15,6))
plt.xlabel('Yearly demand top 10 products')

### Tutorial 5 Calculations of EOQ, weighted moving averages, and smoothed moving averages

#### To calculate yearly EOQ, we start with transposing the pivot table, years as columns and products as rows and reset the index.

In [ ]:
pivot

In [ ]:
pivot=pivot.T
pivot=pivot.reset_index()

In [ ]:
pivot

#### Calculate EOQ by looping over one column at a time

In [ ]:
for i in pivot.columns.tolist()[1:-1]:
    pivot[f'EOQ_{i}']=np.sqrt(pivot[i]*0.05/0.45*2)

In [ ]:
pivot

#### Forecast quantities using 4 year weighted moving averages with weights 0.4, 0.3, 0.2 and 0.1 starting from the most recent year, for the 10 top products

In [ ]:
weights=np.array([0.1,0.2,0.3,0.4])
sum_weights=np.sum(weights)

#### We create a dictionary with elements from the columns list of our top_prod_df and value 0. Add year 2021 to be able to forecast. This dictionary is then added to the original dataframe with the index 2021

In [ ]:
my_dict=dict((el,0) for el in df_top_prod.columns.tolist())
df1 = pd.DataFrame([my_dict], index=['2021'])
df_top_prod=df_top_prod.append(df1)
df_top_prod

#### Loop over the columns to calculate weighted moving averages. 
#### Create a new column and name it with the formatting method weighted_'x', where x is the original column name. 
#### Then the value at each specific cell is smoothed over a window of 4 years, by applying specified weights, with higher weights to the recent years and shifting it 1 cell below so that we get the forecast of the next year

In [ ]:
for i in df_top_prod.columns.tolist()[:10]:
    df_top_prod[f'weighted_{i}']=(df_top_prod[i]).rolling(window=4).apply(lambda x: np.sum(weights*x)/sum_weights,raw=False).shift(1)
df_top_prod

#### To forecast  quantities exponential moving average, similarly to WMA, we once again loop over the columns list.
#### Differently from WMA, EWM, is a built in function in Python, where alfa is calculated as 2/span+1 (look up documentation for more information). 
#### We use adjust False so that it starts on the row we indicate, in this case index 3 and ignores previous rows(0,1,2)

In [ ]:
for i in df_top_prod.columns.tolist():
    df_top_prod[f'ema_{i}']=df_top_prod[i][3:].ewm(span=4,adjust=False).mean().shift(1)

In [ ]:
df_top_prod

#### Export the data to excel

In [ ]:
df_top_prod.to_excel("df_top_prod.xlsx")